KoBERT

In [19]:
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: "'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer": Expected package name at the start of dependency specifier
    'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer
    ^
Hint: = is not a valid operator. Did you mean == ?
'subdirectory'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

c:\Users\31799\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\31799\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\31799\.cache\huggingface\hub\models--skt--kobert-base-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admin

[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [4]:
tokenizer('한국어 모델을 공유합니다.')

{'input_ids': [2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

torch.Size([1, 768])

In [6]:
# 데이터셋클래스
import torch
class OurDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)


In [7]:
# 데이터 로드 및 분할(여기서는 성능상 일부 데이터만 사용)
import pandas as pd
from sklearn.model_selection import train_test_split
# 데이터셋 로드
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
print(len(df))
df = df.sample(frac=0.1)
print(len(df))
X = df.review.tolist()
y = (df.rating >= 6).values.astype(int)
x_,x_test,y_,y_test = train_test_split(X,y,stratify=y,random_state=42,test_size=0.2)
x_train,x_val,y_train,y_val = train_test_split(x_,y_,stratify=y_, random_state=42,test_size=0.2)

14725
1472


In [23]:
from torch.utils.data import DataLoader
# 토큰화
train_input = tokenizer(x_train, truncation=True, padding=True,return_tensors='pt')
val_input = tokenizer(x_val, truncation=True, padding=True, max_length=512,return_tensors='pt')
test_input = tokenizer(x_test, truncation=True, padding=True, max_length=512,return_tensors='pt')
# DataSet 생성
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)
# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=16)
val_loader = DataLoader(val_dataset, batch_size=16,)
test_loader = DataLoader(test_dataset, batch_size=16,)
# KoBERT 한국어 전용 모델 로드
model = BertModel.from_pretrained('skt/kobert-base-v1')
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels):
        super(MyModel, self).__init__()
        self.pretrained_model = pretrained_model
        self.token_size = token_size
        self.num_labels = num_labels
        # 분류기 정의
        self.clf = nn.Linear(self.token_size, self.num_labels)
    def forward(self, inputs):
        outputs = self.pretrained_model(**inputs) # [batch_size, embeddding_dim, num_labels]
        bert_clf_token = outputs.last_hidden_state[:,0,:] # [CLS] 토큰

        return self.clf(bert_clf_token)
model = MyModel(model, token_size=model.config.hidden_size, num_labels=2)
# BERT를 포함한 신경망 모델
# 학습 - 미니배치
  # device
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()
  # optimize
from torch import optim
optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()
  # 학습 스케줄러
from transformers import get_linear_schedule_with_warmup
num_training_steps = len(train_loader)  # 1 epoch에 대한 학습 스텝 수
scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, 
                                            num_warmup_steps=200, 
                                            num_training_steps=num_training_steps)
  # epoch수만큼 loop
total_loss = 0
for batch in train_loader:
  model.train()
  optimizer.zero_grad()
  inputs = {k : v.to(device) for k, v in batch.items() if k != 'labels'}
  labels = batch['labels'].to(device)
  outputs = model(inputs)
  loss = criterion(outputs, labels.long())
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  scheduler.step()
  total_loss += loss.item()
print(total_loss / len(train_loader))

C:\Users\31799\AppData\Local\Temp\ipykernel_17384\1213381581.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}


0.6635477997488894


In [26]:
# val 데이터로 해당 epoch  학습된 모델을 평가
import evaluate
with torch.no_grad():
    metric = evaluate.load('accuracy')
    model.eval()
    for batch in val_loader:
        inputs = {k : v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=-1)
        metric.add_batch(predictions=predictions, references=labels)
    metric.compute()

C:\Users\31799\AppData\Local\Temp\ipykernel_17384\1213381581.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}


In [31]:
df2 = df.reset_index(drop=True)
df2[-5:].review.tolist()

['조금 어설픈듯한 부분도 있었지만 그런 부분을 잊을만큼 충분히 감동적인 영화였다',
 '올해 제가 본 것중 제일 감동적이에요',
 '여배우가 아쉽네요',
 '웅장한 스케일  어마무시한  세계관  의미심장한 결말 .속편을 위한 떡밥을 찾기도 잊은체  영화에 빠져들게 만드는 스토리 개연성 있는 짜임새 전개에 죄강 빌런 타노스의 매력 까지  눈코뜰새 없이 몰입해서 봣네요. 잘차려진 부페인데 어느하나 맛 없는게 없는 영화. 캐릭간의 스토리 편중 공평하고 이만하면  짪은 시간에 그 거대한 세계관을 잘담아 보여줬다 봅니다.  보는내내 심장이 바운스 바운스 ㅋㅋ',
 '순수 평화시위인것으로 위장한 왜곡된 영화~~! 무기고 탈취 총기무장 한 반군 내전 총격전 이였음~~! .']

In [39]:
inputs = tokenizer(df2[-5:].review.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=512)
inputs = {k: v.to(device) for k, v in inputs.items()}
outputs = model(inputs)
preds = torch.softmax(outputs, dim=-1)
preds = torch.argmax(preds, dim=-1)
preds

tensor([1, 1, 1, 1, 1])